In [3]:
import face_recognition
from email.mime import base
from utils import IMAGE_PATH
import numpy as np
import random as rd
import os
import time
import pickle
import utils
import json
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [4]:
N_SAMPLES = 10
PCA_N_COMPONENTS=25
N_NEIGHBORS=10

In [5]:
def get_image(path):
    picture = face_recognition.load_image_file(path)
    return face_recognition.face_encodings(picture)[0]


def generate_C1(set):
    return get_image(set[0]) - get_image(set[1])

def generate_C2(set):
    return get_image(set[0]) - get_image(set[2])

In [37]:
def get_sets():
    with open("test_set.json") as f:
        individuals = json.load(f)

    C1 = []
    C2 = []

    count = 0
    for dir, set in individuals.items():
        # print("Generating C1 for {}...".format(dir))
        C1.append(generate_C1(set))
        # print("Generating C2 for {}...".format(dir))
        C2.append(generate_C2(set))
        if count == 50: break
    return[*C1, *C2], [*[1 for individual in C1], *[0 for individual in C2]]

In [58]:
def new_train_svm(X, y, kernel='rbf', C=0.5):
    with open("train_set.json") as f:
        individuals = json.load(f)

    clf = svm.SVC(kernel=kernel, C=C)
    
    # print(f"Fitting {kernel} kernel...")
    start = time.time()
    clf.fit(X, y)
    end = time.time()
    # print("Fitted in {} seconds".format(end - start))
    
    return clf

In [49]:
def new_train_knn_pca(n_components=PCA_N_COMPONENTS,k=N_NEIGHBORS):
    with open("train_set.json") as f:
        individuals = json.load(f)

    C1 = []
    C2 = []

    for dir, set in individuals.items():
        print("Generating C1 for {}...".format(dir))
        C1.append(generate_C1(set))
        print("Generating C2 for {}...".format(dir))
        C2.append(generate_C2(set))
    
    X = [*C1, *C2]

    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    pca = PCA(n_components=n_components)
    pca.fit(X)
    X = pca.transform(X)
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X, [*[1 for individual in C1], *[0 for individual in C2]])
    return knn

In [ ]:
X, y = get_sets()
clf = new_train_svm(X, y)

In [40]:
def test_svm(clf):
    with open("test_set.json") as f:
            individuals = json.load(f)

    switch = 0
    correct = 0
    for dir, set in individuals.items():
        if (switch):
            res = clf.predict([get_image(set[0]) - get_image(set[1])])
            if (res[0] == np.int64(1)): correct +=1
            # print(type(res[0]))
            switch = 0
        else:
            res = clf.predict([get_image(set[0]) - get_image(set[2])])
            if (res[0] == np.int64(0)): correct +=1
            # print(res[0])
            switch = 1
    accuracy = correct / len(individuals)
    # print("Accuracy = {}".format(accuracy))
    return accuracy


In [55]:
def train(kernel, C_values, X, y):
    accuracy = -1
    # while accuracy < 0.9:
    for C in C_values:
        clf = new_train_svm(X, y, kernel, C)
        accuracy = test_svm(clf)
        print("Accuracy = {}".format(accuracy))
        print("C = {}".format(C))
        print("Kernel = {}".format(kernel))

In [60]:
X, y = get_sets()

In [50]:
C_values = [0.01, 0.1, 1, 10, 100, 1000]

In [61]:
kernels = ['poly', 'rbf', 'sigmoid', 'linear']
for kernel in kernels:
    train(kernel, C_values, X, y)

Accuracy = 0.56
C = 0.01
Kernel = poly
Accuracy = 0.59
C = 0.1
Kernel = poly
Accuracy = 1.0
C = 1
Kernel = poly
Accuracy = 1.0
C = 10
Kernel = poly
Accuracy = 1.0
C = 100
Kernel = poly
Accuracy = 1.0
C = 1000
Kernel = poly
Accuracy = 1.0
C = 0.01
Kernel = rbf
Accuracy = 1.0
C = 0.1
Kernel = rbf
Accuracy = 1.0
C = 1
Kernel = rbf
Accuracy = 1.0
C = 10
Kernel = rbf
Accuracy = 1.0
C = 100
Kernel = rbf
Accuracy = 1.0
C = 1000
Kernel = rbf
Accuracy = 0.68
C = 0.01
Kernel = sigmoid
Accuracy = 0.68
C = 0.1
Kernel = sigmoid
Accuracy = 0.7
C = 1
Kernel = sigmoid
Accuracy = 0.52
C = 10
Kernel = sigmoid
Accuracy = 0.48
C = 100
Kernel = sigmoid
Accuracy = 0.45
C = 1000
Kernel = sigmoid
Accuracy = 0.68
C = 0.01
Kernel = linear
Accuracy = 0.68
C = 0.1
Kernel = linear
Accuracy = 0.73
C = 1
Kernel = linear
Accuracy = 0.74
C = 10
Kernel = linear
Accuracy = 0.83
C = 100
Kernel = linear
Accuracy = 0.89
C = 1000
Kernel = linear


In [ ]:
knn = new_train_knn_pca()

In [ ]:
with open("test_set.json") as f:
        individuals = json.load(f)

correct = 0
X = get_test_sets()
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
pca = PCA(n_components=PCA_N_COMPONENTS)
pca.fit(X)
X = pca.transform(X)
count = 0
for set in X:
    if (count < 50):
        res = knn.predict([set])
        if (res[0] == np.int64(1)): correct +=1
        count += 1
    else:
        res = knn.predict([set])
        if (res[0] == np.int64(0)): correct +=1


print("Accuracy = {}".format(correct/len(individuals)))


In [ ]:
knn.predict([get_image("images/00911/00911fa010_960530.tif") - get_image("images/00510/00510fa010h_940519.tif")])